In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import sys
import pickle
import skimage
from imaris_ims_file_reader.ims import ims
import h5py
import tables
np.random.seed(42) # for reproducibility

In [3]:
sys.path.insert(1, 'aics-shparam')
from aicsshparam import shtools, shparam

# example

In [2]:
# Function that returns binary images containing one of the three
# shapes: cubes, spheres octahedrons of random sizes.
def get_random_3d_shape():
    idx = np.random.choice([0, 1, 2], 1)[0]
    element = [ball, cube, octahedron][idx]
    label = ['ball', 'cube', 'octahedron'][idx]
    img = element(10 + int(10 * np.random.rand()))
    img = np.pad(img, ((1, 1), (1, 1), (1, 1)))
    img = img.reshape(1, *img.shape)
    # Rotate shapes to increase dataset variability.
    img = shtools.rotate_image_2d(
        image=img,
        angle=360 * np.random.rand()
    ).squeeze()
    return label, img

# Compute spherical harmonics coefficients of shape and store them
# in a pandas dataframe.
df_coeffs = pd.DataFrame([])
for i in range(30):
    # Get a random shape
    label, img = get_random_3d_shape()
    # Parameterize with L=4, which corresponds to50 coefficients
    # in total
    (coeffs, _), _ = shparam.get_shcoeffs(image=img, lmax=4)
    coeffs.update({'label': [label]})
    coeffs = pd.DataFrame.from_dict(coeffs)
    df_coeffs = pd.concat([df_coeffs,coeffs], ignore_index=True)

# Vizualize the resulting dataframe
with pd.option_context('display.max_rows', 5, 'display.max_columns', 5):
    display(df_coeffs)

,shcoeffs_L0M0C,shcoeffs_L0M1C,...,shcoeffs_L4M4S,label
0,11.812591,0.0,...,-0.528787,octahedron
1,14.995109,0.0,...,-0.012269,ball
...,...,...,...,...,...
28,11.811053,0.0,...,0.735944,octahedron
29,11.147437,0.0,...,0.635403,octahedron


# progeria cell

In [4]:
tables.file._open_files.close_all()
fn = "/Users/amulya/Downloads/Late_progeria_IF_Test_1.ims"
f = ims(fn,write=True)
c1 = f[0][0]
c2 = f[0][1]

Opening writeable file: /Users/amulya/Downloads/Late_progeria_IF_Test_1.ims 



/usr/local/lib/python3.11/site-packages/imaris_ims_file_reader/ims.py:189: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  self.hf = h5py.File(self.filePathComplete, 'a', swmr=True)


## binary mask

In [9]:
sys.path.insert(1, 'aics-segmentation')
# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_2d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_3d
from aicssegmentation.core.utils import get_middle_frame, hole_filling, get_3dseed_from_mid_frame
from skimage.morphology import remove_small_objects,  dilation, ball
from skimage.segmentation import watershed

In [12]:
intensity_scaling_param = [4000]
# intensity normalization
struct_img = intensity_normalization(c2, scaling_param=intensity_scaling_param)

intensity norm: min-max norm with upper bound 4000


In [16]:
gaussian_smoothing_sigma = 8
structure_img_smooth = image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)

In [63]:
from skimage.exposure import histogram
from skimage.feature import canny
import scipy
from skimage.morphology import closing, square
from skimage.color import label2rgb

binary_mask = np.zeros(np.shape(c2))
# 
for i in range(72,np.shape(c2)[0]):
    print(i)
    thresh = skimage.filters.threshold_otsu(structure_img_smooth[i])
    binary = structure_img_smooth[i] > thresh
    bw = closing(binary, square(5))
    labeled,_ = scipy.ndimage.label(bw)
    region_table = pd.DataFrame.from_dict(skimage.measure.regionprops_table(labeled,properties=['label','area']))
    largest = region_table.loc[np.argmax(region_table['area']),'label']
    binary_mask[i][np.where(labeled==largest)] = 1
    image_label_overlay = label2rgb(binary_mask[i], image=structure_img_smooth[i], bg_label=0)
    #plt.imshow(image_label_overlay)
    #plt.show()

72
73
74
75
76
77
78
79
80
81
82
83
84
85
86


In [ ]:
att = np.array(binary_mask[0],dtype=int)
att_2 = skimage.morphology.diameter_opening(att, diameter_threshold=20)
plt.imshow(att_2,cmap="gray")

In [65]:
skimage.io.imsave("Late_progeria_IF_Test_1_c2_binarymask_v2.tif", binary_mask)

## LMNA stain

In [71]:
(coeffs, _), (image_, mesh, grid_down, transform) = shparam.get_shcoeffs(image=binary_mask, lmax=16)

In [ ]:
coeffs = pd.DataFrame.from_dict({k : [v] for k, v in coeffs.items()})

In [72]:
with open("progeria_test_binary_v2_coefs_l16.pkl", "wb") as f:
    pickle.dump(coeffs, f)

## visualizations

In [68]:
# orig cell mesh
mesh_orig, img_output, centroid = shtools.get_mesh_from_image(image=binary_mask)

In [73]:
shtools.save_polydata(mesh, 'mesh_calc_binary_l16_v2.vtk')

In [70]:
shtools.save_polydata(mesh_orig, 'mesh_orig_binary_v2.vtk')